In [ ]:
import numpy as np
from scipy.stats import kurtosis, skew

# Function to extract features from any given input
def extract_features(cipher_text):
    try:
        cipher_bytes = bytes.fromhex(cipher_text)
        cipher_array = np.frombuffer(cipher_bytes, dtype=np.uint8)

        # Calculate statistical features
        mean = np.mean(cipher_array)
        std_dev = np.std(cipher_array)

        # Histogram for entropy calculation
        histogram, _ = np.histogram(cipher_array, bins=256, range=(0, 256))
        histogram = histogram / np.sum(histogram)  # Normalize histogram
        entropy = -np.sum(histogram * np.log2(histogram + 1e-10))

        # Kurtosis and skewness
        kurt = kurtosis(cipher_array)
        skewness = skew(cipher_array)

        # Min, max, and range
        min_value = np.min(cipher_array)
        max_value = np.max(cipher_array)
        range_value = max_value - min_value

        # Number of unique byte values
        unique_byte_count = len(np.unique(cipher_array))

        # Combine features into a single list
        features = [
            mean, std_dev, entropy, kurt, skewness, min_value, max_value, range_value,
            unique_byte_count
        ]

        return features

    except Exception as e:
        print("Error extracting features from input.")
        print(e)
        return None

from tensorflow.keras.models import load_model
import joblib

# Load models
hash_model = load_model('/content/drive/My Drive/hash_model/hashing_algorithm_cnn_model.h5')
stream_model = load_model('/content/drive/My Drive/stream_model/stream_algorithm_cnn_model.h5')
block_model = load_model('/content/drive/My Drive/block_model/block_algorithm_cnn_model.h5')

# Load scalers and encoders
hash_scaler = joblib.load('/content/drive/My Drive/hash_model/scaler.pkl')
hash_label_encoder = joblib.load('/content/drive/My Drive/hash_model/label_encoder.pkl')

stream_scaler = joblib.load('/content/drive/My Drive/stream_model/scaler.pkl')
stream_label_encoder = joblib.load('/content/drive/My Drive/stream_model/label_encoder.pkl')

block_scaler = joblib.load('/content/drive/My Drive/block_model/scaler.pkl')
block_label_encoder = joblib.load('/content/drive/My Drive/block_model/label_encoder.pkl')

def preprocess_and_predict(cipher_text):
    # Extract features
    features = extract_features(cipher_text)
    features = np.array(features).reshape(1, -1)

    # Predictions for Hash Model
    hash_features = hash_scaler.transform(features)
    hash_features = hash_features.reshape(1, 9, 1, 1)
    hash_prediction = hash_model.predict(hash_features)
    hash_predicted_label_index = np.argmax(hash_prediction, axis=1)
    hash_predicted_label = hash_label_encoder.inverse_transform(hash_predicted_label_index)[0]
    hash_confidence = hash_prediction[0][hash_predicted_label_index[0]]

    # Predictions for Stream Model
    stream_features = stream_scaler.transform(features)
    stream_features = stream_features.reshape(1, 9, 1, 1)
    stream_prediction = stream_model.predict(stream_features)
    stream_predicted_label_index = np.argmax(stream_prediction, axis=1)
    stream_predicted_label = stream_label_encoder.inverse_transform(stream_predicted_label_index)[0]
    stream_confidence = stream_prediction[0][stream_predicted_label_index[0]]

    # Predictions for Block Model
    block_features = block_scaler.transform(features)
    block_features = block_features.reshape(1, 9, 1, 1)
    block_prediction = block_model.predict(block_features)
    block_predicted_label_index = np.argmax(block_prediction, axis=1)
    block_predicted_label = block_label_encoder.inverse_transform(block_predicted_label_index)[0]
    block_confidence = block_prediction[0][block_predicted_label_index[0]]

    # Majority Vote Mechanism
    predictions = {
        'Stream Cipher Model': (stream_predicted_label, stream_confidence),
        'Hash Model': (hash_predicted_label, hash_confidence),
        'Block Cipher Model': (block_predicted_label, block_confidence)
    }

    prediction_counts = {}
    for model, (label, confidence) in predictions.items():
        if label in prediction_counts:
            prediction_counts[label] += 1
        else:
            prediction_counts[label] = 1

    final_prediction = max(prediction_counts, key=prediction_counts.get)

    return final_prediction, predictions

# Example cipher text for prediction
cipher_text = '08 cb ff 8c cd ec b5 56 25 b0 b0'  # Replace with your input

final_prediction, model_predictions = preprocess_and_predict(cipher_text)

print(f"Predicted Algorithm (Majority Vote): {final_prediction}\n")
print("Model Predictions:")
for model_name, (label, confidence) in model_predictions.items():
    print(f" - {model_name}: {label} (Confidence: {confidence:.4f})")



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 273ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 268ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 802ms/step
Predicted Algorithm (Majority Vote): RC4

Model Predictions:
 - Stream Cipher Model: RC4 (Confidence: 0.9995)
 - Hash Model: SHA256 (Confidence: 0.9995)
 - Block Cipher Model: 3DES (Confidence: 0.4679)
